# Визуализация сложных данных

## Введение

В руководствах по визуализации данных обычно используются предварительно обработанные данные. Но как насчет наборов данных в дикой природе? Что делать с отсутствующии данными? Или выбросами, которые в значительной степени искажают визуализацию ? Что делать, когда слишком много наблюдений для интерпретации в диаграмме рессеяния? В этой стать будут представлены некоторые методы, которые мы можем использовать для решения этих проблем.

Допусти, мы новые агенты по недвижимости, которые хотят использовать данные, чтобы лучше понять связь между ценой и кол-ом спален в доме. Мы будет использовать набор данных который мы вызвали *`housing`* из [Kaggle](https://www.kaggle.com/austinreese/usa-housing-listings) на USA Housing Listings.

## Отсутствующие данные

Неполные наблюдения - или отсутствующие данные -обычно игнорируются функциями построения графиков в широко используемых библиотеках Python, таких как *`matplotlib`* и *`seaborn`*. Поэтому мы можем захотеть удалить эти строки или подставить отсутствующие значения перед построение графика. Мы можем проверить отсутствующие данные с помощью *`.info()`*

На основании полученных данных мы можем быть обеспокоены некоторыми столбцами как например *`laundry_options`*, послькольку *`parking_options`* в них больше пропущенных значений, чем в других столбцах.

## Предварительный вид

Давайте сначала рассмотрим две переменные и посмотрим, с какими проблемами мы сталкиваемся. Вот график цены и площади в квадратных футах

![](./img/3_3_1.png)

Не похоже, что на этом графике много точек, хотя их должно быть более 300 000. *`1e6`* и *`1e9`* на осях X и Y соответственно указывают на то, что масштаб и диапазон для обеих функций невероятно велики. Например, у нас есть по крайней мере один листинг жилья, который стоит почти 3 000 000 000 долларов в месяц. Первое, что нам придется сделать, чтобы более эффективно визуализировать данные, - это разобраться с этими выбросами.

## Построение графика с выбросами

Мы можем сократить каждую функцию на графике, чтобы вырезать выбросы, пока не получим лучшее представление о данных. Может потребоваться некоторое кол-во проб и ошибок, чтобы найти правильные значения, поэтому давайте начнем с ограничения *`price`* до менее чем 10 000 000 и *`sqfeet`* менее чем 2 000 000.

```python
housing2 = housing[(housing.price < 10000000) & (housing.price>0)]
housing2 = housing2[(housing2.sqfeet < 2000000) & (housing2.sqfeet>0)]

sns.scatterplot(housing2['sqfeet'], housing2['price'])
```

![](./img/3_3_2.png)

Эта диаграмма рассеивания немного лучше. Мы видим больше точек, отображаемых в нижней левой части диаграммы. Давайте приблизимся к этому кластеру точек: давайте ограничим *`price`* и *`sqfeet`* значениями менее 20 000

```python
housing2 = housing[(housing.price < 20000) & (housing.price>0)]
housing2 = housing2[(housing2.sqfeet < 20000) & (housing2.sqfeet>0)]

sns.scatterplot(housing2['sqfeet'], housing2['price'])
```

![](./img/3_3_3.png)

Теперь мы начинаем видеть все точки! С правой стороны все еще много пустого пространства, поэтому давайте ограничим наши данные еще раз, на этот раз ограничив и *`price`* и *`sqfeet`* значениями менее 3000

```python
## limit price and sqfeet to < 3000
housing2 = housing[(housing.price < 3000) & (housing.price>0)]
housing2 = housing2[(housing2.sqfeet < 3000) & (housing2.sqfeet>0)]

sns.scatterplot(housing2['sqfeet'], housing2['price'])
```

![](./img/3_3_4.png)

Теперь мы действительно можем видеть большую часть точек из нашего набора данных. Однако здесь все еще так много точек, что они все напечатаны друг на друге. Это означает, что мы не можем визуализировать плотность точек и, следовательно, общую взаимосвязь между ценой и площадью.

## Визуализация множества точек данных

Когда точек данных слишком много для визуализации, мы можем взять случайное подмножество данных. Это будет означать меньше точек, и поскольку это случайное подмножество, его ве равно можно будет приблизительно обощить на весь набор данных. Давайте попробуем использовать случайные 5% данных

```python
perc = 0.05
housing_sub = housing2.sample(n = int(housing2.shape[0]*perc))

sns.scatterplot(housing_sub['sqfeet'], housing_sub['price'])
```

![](./img/3_3_5.png)

По-прежнему много совпадений, но мы фактически можем видеть положительную линейную связь между площадью и ценой, которую изначально было трудно визуализировать.

Мы все еще можем это улучшить. Мы можем попробовать сделать каждую точку меньше, чтобы лучше видеть места с большей концентрацией нанесенных точек:

```python
sns.scatterplot(housing_sub['sqfeet'], housing_sub['price'], s = 5)
```

![](./img/3_3_6.png)

Этот график лучше предыдущего, потому что, с первого взгляда, мы можем увидеть более высокую концентрацию точек в *`sqfeet`* в диапазоне от 500 до 1500 и в диапазоне от 500 до 2000. *`price`* однако это все еще не дает нам хорошего понимания того, сколько точек находится в этом среднем кластере. Вместо того, чтобы рисовать точки меньше, мы можем сделать их более прозрачными. Таким образом, мы можем интерпретировать интенсивность цвета, чтобы понять перекрытие

```python
sns.scatterplot(housing_sub['sqfeet'], housing_sub['price'], alpha = 0.2)
```

![](./img/3_3_7.png)

Мы видим, что нижняя часть графика темнее верхней. Это связано с тем, что на нижних *`price`* уровнях накладываются друг на друга гораздо больше точек, а по мере *`price`* увеличения точек становится меньше

Мы также могли бы рассмотреть возможность построения LOWESS (Locally Weighted Scatterplot Smoothing) сглаживание по нашим точкам данных. Это нарисует линию через приблизительную среднюю цену для каждого значения *`sqfeet`*

```python
sns.lmplot(x='sqfeet', y='price', data = housing_sub, line_kws={'color': 'black'}, lowess=True)
```

![](./img/3_3_8.png)

Хотя отдельные точки сложнее читать, линия дает нам информацию о взаимосвязи между этими двумя характеристиками

## Визуализация дискретных переменных

Допустим, мы хотим посмотреть на связь между *`beds`* и *`baths`* в нашем наборе данных. Мы можем легко построить диаграмму рассеивания

```python
sns.scatterplot('beds', 'baths', data = housing_sub)
```

![](./img/3_3_9.png)

Хотя этот график показывает нам каждую комбинаю кол-ва кроватей и ванных комнат в нашем наборе данных, он не показывает нам, сколько наблюдений имеется. Это связано с тем, что обе функции являются *`дискретными`* значениями, в данном случае это означает ограничение целыми числами для *`beds`* и половинными числами для *`bath`*. Таким образом, каждая точка данных, представляющая 3 кровати и 2 ванные комнаты, отображается в том же масте, что и другие, идеально перекрываясь с выглядя как одна точка.

Добавлние *`дрожания`* корректирует распределение точек по одной (или обеим) осям, чтобы было легче увидеть множество точек в каждой группе

```python
sns.lmplot('beds', 'baths', data = housing_sub, x_jitter = .15, y_jitter = .15, fit_reg = False)
```

![](./img/3_3_10.png)

*`bath`* мы можем посмотреть на этот график и узнать гораздо больше, что предыдущий. Например, мы знаем, что на каждом уровне меньше точек, когда *`beds`* равно 6, по сравнения с 5

## Логарифмическое преобразование

Иногда, когда данные находятся в *`логарифмической шкале`*, может быть сложно визуализировать распределение значений. Признаки с положительными значениями, которые сильно смещены впрво, являются основными кандидатами на логарифмическое преобразование. Давайте посмотрим на распределение *`price`* из нашего набора данных

```python
sns.displot(housing.price)
```

![](./img/3_3_11.png)

Здесь мы видим один высокий пик с левой стороны и очень длинный правый хвост дволь оси Х. Хотя мы могли бы попытаться сократить *`price`* значения, как и раньше, было бы полезно вместо этого попытаться построить распределение логарифмической цены

```python
log_price = housing.price[housing.price>0]
log_price = np.log(log_price)
sns.displot(log_price)
plt.xlabel('log_price')
```

![](./img/3_3_12.png)

Эта гистограмма предоставляет гораздо больше информации, чем данные в исходной форме. Мы даже можем ограничить график только между 5 и 10, чтобы увидеть распределение более четко

```python
sns.displot(log_price)
plt.xlabel('log price')
plt.xlim(5,10)
```

![](./img/3_3_13.png)

Этот график покаывает, что логарифмическая цена является унимодальной и приблизительно нормально распределенной. Это полезное знание, если мы хотим построить модель для прогнозирования цен в будущем.

# Заключение

Создание интерпретируемых визуализаций данных не всегда так просто, как простое нанесение всех данных на график. Часто визуализации требуют дополнительных шагов, таких как дрожание, уменьшение или увеличение непрозрачности точек или преобразование данных. Выполнение этих шагов поможет вам в будущем создавать более динамичные и интерпретируемые визуализации.